In [ ]:
import xarray as xr

In [2]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(scheduler_port=8789)
cluster

In [3]:
# from distributed import LocalCluster
# cluster = LocalCluster(n_workers=3,processes=3,threads_per_worker=1,)
# cluster

In [3]:
from distributed import Client
# client=Client(cluster)
client=Client('tcp://localhost:8789')
client

Client Scheduler: tcp://localhost:8789 Dashboard: http://localhost:8787/status,Cluster Workers: 40 Cores: 80 Memory: 600.00 GB


And open all the files as a multi-file dataset with XArray, and set some chunks that will be used by the Zarr library to save separate binary chunks. The rule-of-thumb for data that is going to S3 is roughly 10M (uncompressed?) per chunk

In [4]:
import glob
files=glob.glob('/scratch/pawsey0106/pbranson/aqua_files/*')
files=sorted(files)
ds = xr.open_mfdataset(files,chunks={'longitude':1223,'latitude':856},concat_dim='time',parallel=True)
ds = ds.chunk({'time':7,'longitude':1223,'latitude':856})
ds

<xarray.Dataset>
Dimensions:    (latitude: 7001, longitude: 10001, time: 4882)
Coordinates:
  * latitude   (latitude) float64 10.0 9.99 9.98 9.97 ... -59.98 -59.99 -60.0
  * longitude  (longitude) float64 80.0 80.01 80.02 80.03 ... 180.0 180.0 180.0
  * time       (time) datetime64[ns] 2002-07-06 2002-07-07 ... 2016-12-31
Data variables:
    K_490      (time, latitude, longitude) float32 dask.array<shape=(4882, 7001, 10001), chunksize=(7, 856, 1223)>
Attributes:
    history:      File initialised at 2015-12-16T15:54:44.125153\nInitialised...
    Conventions:  CF-1.6
    source_path:  imos-srs/archive/oc/aqua/1d/v201508/2002/07/A20020706.L2OC_...

In [5]:
print('Dataset size %.1f GB' % (ds.nbytes/1E9)) # Because australia is a big 'hole' in the middle, 1.4TB compresses down to 180GB on disk

Dataset size 1367.3 GB


In [6]:
import zarr
import os
zarr_path = os.environ['MYSCRATCH'] + '/aqua.P1D.aust.K_490.Full.zarr' # MYSCRATCH is a Pawsey HPC environment variable

compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)
encoding = {vname: {'compressor': compressor} for vname in ds.variables}
ds.to_zarr(zarr_path,mode='w',encoding=encoding)

In [8]:
ds = xr.open_zarr(zarr_path)
ds

<xarray.Dataset>
Dimensions:    (latitude: 7001, longitude: 10001, time: 4882)
Coordinates:
  * latitude   (latitude) float64 10.0 9.99 9.98 9.97 ... -59.98 -59.99 -60.0
  * longitude  (longitude) float64 80.0 80.01 80.02 80.03 ... 180.0 180.0 180.0
  * time       (time) datetime64[ns] 2002-07-06 2002-07-07 ... 2016-12-31
Data variables:
    K_490      (time, latitude, longitude) float32 dask.array<shape=(4882, 7001, 10001), chunksize=(7, 856, 1223)>
Attributes:
    Conventions:  CF-1.6
    history:      File initialised at 2015-12-16T15:54:44.125153\nInitialised...
    source_path:  imos-srs/archive/oc/aqua/1d/v201508/2002/07/A20020706.L2OC_...